In [ ]:
import pandas as pd
import os
from datetime import datetime


In [ ]:
def getQuarter(x):
    months=x.values
    quarters=[]
    for month in months:
        if month>=1 and month<=3:
            quarters.append(1)
        elif month>=4 and month<=6:
            quarters.append(2)
        elif month>=7 and month<=9:
            quarters.append(3)
        elif month>=10 and month<=12:
            quarters.append(4)

    return quarters

In [ ]:
year=2023
radius=500

In [ ]:
df=pd.read_csv('U:\\Projects\\Huq\\Faraz\\final_OD_work\\2023\\trips\\temp_trip_500m_2023.csv',parse_dates=['org_arival_time','org_leaving_time','dest_arival_time'])
df

In [ ]:
df=df.assign(trip_id=lambda df: df.groupby(['uid'])['trip_time'].transform(lambda x: [i for i in range(1,len(x)+1)]))


df=df[['uid','trip_id', 'org_lat', 'org_lng', 'org_arival_time', 'org_leaving_time',
'dest_lat', 'dest_lng', 'dest_arival_time', 'stay_points',
'trip_points', 'trip_time', 'stay_duration', 'observed_stay_duration', 
'origin_oa_id','destination_oa_id', 'origin_dz_id','destination_dz_id', 'origin_iz_id', 'destination_iz_id',
'origin_council_area_id','destination_council_area_id', 'origin_council_area_name','destination_council_area_name']]

df

In [ ]:
#############################################################
#                                                           #
#                    Calculate Total Trips/User             #
#                                                           #
#############################################################

print(f'{datetime.now()}: Calculating Total Trips/User')
df['month']=df['org_leaving_time'].dt.month
df=df.assign(total_trips=lambda df: df.groupby('uid')['trip_id'].transform(lambda x: len(x)))
df['quarter']=df.groupby(['uid','month'])['month'].transform(getQuarter)

df.drop(columns=['month'],inplace=True)

print(f'{datetime.now()}: Trips/User Calculated')

In [ ]:
df['total_trips'].describe()

In [ ]:
df[df['total_trips']>1000]['uid'].nunique()

In [ ]:
print(f'{datetime.now()}: Calculating TPAD')
active_day_df=pd.read_csv(f'U:\\Projects\\Huq\\Faraz\\final_OD_work\\{year}\\active_days_stat_{year}.csv')
df=(
    df.merge(active_day_df,how='left',left_on='uid',right_on='uid')
    .assign(tpad=lambda tdf: tdf['total_trips']/tdf['total_active_days'])
    )
print(f'{datetime.now()}: TPAD Calculated')

In [ ]:
df.head()

In [ ]:
df['tpad'].describe()

In [ ]:
df[df['total_trips']==651]